In [1]:
import sys
import importlib
sys.path.append('./rdd-py')
import rdd

In [2]:
r = rdd.parse_rds("rds/integer1234.rds")
r.object

[2024-07-23 15:58:35,581][INFO] Reading rds/integer1234.rds
[2024-07-23 15:58:35,587][INFO] Done reading rds/integer1234.rds


(
.value: [1,2,3,4]
.attrs: {
..sexptype: INTSXP}
)

In [3]:
r = rdd.parse_rds("rds/mtcars.rds")
r.object

[2024-07-23 15:58:37,267][INFO] Reading rds/mtcars.rds
[2024-07-23 15:58:37,272][INFO] Done reading rds/mtcars.rds


(
.value: [
..(
...value: [21.0,21.0,22.8,21.4,18.7,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [6.0,6.0,4.0,6.0,8.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [160.0,160.0,108.0,258.0,360.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [110.0,110.0,93.0,110.0,175.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [3.9,3.9,3.85,3.08,3.15,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [2.62,2.875,2.32,3.215,3.44,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [16.46,17.02,18.61,19.44,17.02,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [0.0,0.0,1.0,1.0,0.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [1.0,1.0,1.0,0.0,0.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [4.0,4.0,4.0,3.0,3.0,...]
...attrs: {
....sexptype: REALSXP}
..),
..(
...value: [4.0,4.0,1.0,1.0,2.0,...]
...attrs: {
....sexptype: REALSXP}
..)
.]
.attrs: {
..names:(
...value: [mpg,cyl,disp,hp,drat,...]
...at

In [5]:
r = rdd.parse_rds("rds/seurat.rds")
r.object

[2024-07-23 15:59:32,375][INFO] Reading rds/seurat.rds
[2024-07-23 15:59:35,351][INFO] Done reading rds/seurat.rds


(
.value: (
..value: {
...assays:(
....value: [
.....(
......value: (
.......value: {
........layers:(
.........value: [
..........(
...........value: (
............value: {
.............i:(
..............value: [30,59,86,97,113,...]
..............attrs: {
...............sexptype: INTSXP}
.............),
.............p:(
..............value: [0,1819,3557,4111,5907,...]
..............attrs: {
...............sexptype: INTSXP}
.............),
.............Dim:(
..............value: [36601,972]
..............attrs: {
...............sexptype: INTSXP}
.............),
.............Dimnames:(
..............value: [
...............(
................value: 
................attrs: {
.................sexptype: NILSXP}
...............),
...............(
................value: 
................attrs: {
.................sexptype: NILSXP}
...............)
..............]
..............attrs: {
...............sexptype: VECSXP}
.............),
.............x:(
..............value: [1.0,1.0,1.0,1.0,1.0,.

# Seurat 2 anndata

In [123]:
import scanpy as sc
import numpy as np
import pandas as pd
from scipy import sparse
import anndata

## obs

In [83]:
metadata = r.object.value.value[1]
metadata

['meta.data',
 (
 .value: [
 ..(
 ...value: [1,1,1,1,1,...]
 ...attrs: {
 ....levels:(
 .....value: [SeuratProject]
 .....attrs: {
 ......sexptype: STRSXP}
 ....),
 ....class:(
 .....value: [factor]
 .....attrs: {
 ......sexptype: STRSXP}
 ....),
 ....sexptype: INTSXP}
 ..),
 ..(
 ...value: [5029.0,4343.0,898.0,4999.0,601.0,...]
 ...attrs: {
 ....sexptype: REALSXP}
 ..),
 ..(
 ...value: [1819,1738,554,1796,416,...]
 ...attrs: {
 ....sexptype: INTSXP}
 ..)
 .]
 .attrs: {
 ..row.names:(
 ...value: [AAACCTGCAGCCTGTG-1,AAACGGGTCGCTGATA-1,AAAGATGTCCTCGCAT-1,AAAGCAAAGTATGACA-1,AAAGCAATCCATTCTA-1,...]
 ...attrs: {
 ....sexptype: STRSXP}
 ..),
 ..names:(
 ...value: [orig.ident,nCount_RNA,nFeature_RNA]
 ...attrs: {
 ....sexptype: STRSXP}
 ..),
 ..class:(
 ...value: [data.frame]
 ...attrs: {
 ....sexptype: STRSXP}
 ..),
 ..sexptype: VECSXP}
 )]

In [122]:
# first columns is factor:
obs_names = metadata[1].attributes[0][1].value
colnames = metadata[1].attributes[1][1].value
cols = {}
levels = metadata[1].value[0].attributes[0][1].value
cols[colnames[0]] = pd.Series(levels)[pd.Series(metadata[1].value[0].value)-1]
cols[colnames[1]] = metadata[1].value[1].value
cols[colnames[2]] = metadata[1].value[2].value
obs = pd.DataFrame(cols)
obs.index = obs_names
obs

,orig.ident,nCount_RNA,nFeature_RNA
AAACCTGCAGCCTGTG-1,SeuratProject,5029.0,1819
AAACGGGTCGCTGATA-1,SeuratProject,4343.0,1738
AAAGATGTCCTCGCAT-1,SeuratProject,898.0,554
AAAGCAAAGTATGACA-1,SeuratProject,4999.0,1796
AAAGCAATCCATTCTA-1,SeuratProject,601.0,416
...,...,...,...
TTTGGTTTCAACGAAA-1,SeuratProject,1408.0,815
TTTGGTTTCTGAGGGA-1,SeuratProject,6074.0,2003
TTTGTCAAGCCCAATT-1,SeuratProject,8067.0,2727
TTTGTCAAGTCTCAAC-1,SeuratProject,4770.0,1767


In [31]:
X = r.object.value.value[0][1].value[0].value.value[0][1]
X

(
.value: [
..(
...value: (
....value: {
.....i:(
......value: [30,59,86,97,113,...]
......attrs: {
.......sexptype: INTSXP}
.....),
.....p:(
......value: [0,1819,3557,4111,5907,...]
......attrs: {
.......sexptype: INTSXP}
.....),
.....Dim:(
......value: [36601,972]
......attrs: {
.......sexptype: INTSXP}
.....),
.....Dimnames:(
......value: [
.......(
........value: 
........attrs: {
.........sexptype: NILSXP}
.......),
.......(
........value: 
........attrs: {
.........sexptype: NILSXP}
.......)
......]
......attrs: {
.......sexptype: VECSXP}
.....),
.....x:(
......value: [1.0,1.0,1.0,1.0,1.0,...]
......attrs: {
.......sexptype: REALSXP}
.....),
.....factors:(
......value: []
......attrs: {
.......sexptype: VECSXP}
.....),
.....class:(
......value: [dgCMatrix]
......attrs: {
.......package:(
........value: [Matrix]
........attrs: {
.........sexptype: STRSXP}
.......),
.......sexptype: STRSXP}
.....)
....}
....attrs: {
.....sexptype: LISTSXP}
...)
...attrs: {
....sexptype: S4SXP}
..)


In [82]:
i = X.value[0].value.value[0][1].value
p = X.value[0].value.value[1][1].value
dim = X.value[0].value.value[2][1].value
x = X.value[0].value.value[4][1].value
XX = sparse.csc_matrix((x, i, p), dim)
XX

<36601x972 sparse matrix of type '<class 'numpy.float64'>'
	with 1798509 stored elements in Compressed Sparse Column format>

In [125]:
var = pd.DataFrame({'gene_name': r.object.value.value[0][1].value[0].value.value[2][1].attributes[2][1].value[0].value})
var.index = var.gene_name
var

,gene_name
gene_name,
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC141272.1,AC141272.1
AC023491.2,AC023491.2
AC007325.1,AC007325.1


In [128]:
adata = anndata.AnnData(X=XX.T,obs=obs,var=var)
adata

AnnData object with n_obs × n_vars = 972 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA'
    var: 'gene_name'